In [2]:
import pandas as pd
from pathlib import Path

src = Path("blast/blast.accession")
out = Path("tables/accession_taxid.parquet")  # put in repo if it's small enough

ia = pd.read_table(
    src,
    index_col=0,
    names=["name", "length", "taxid"],
)

# clean accession index
ia = ia.loc[~ia.index.isna()].copy()
ia.index = ia.index.astype(str)
ia = ia[~ia.index.duplicated(keep="first")]

# normalize taxid to first component
ia["taxid1"] = ia["taxid"].astype(str).str.split(";").str[0]
ia = ia[ia["taxid1"].str.fullmatch(r"\d+")]
ia["taxid1"] = ia["taxid1"].astype("int32")

# keep only what you need
acc2tax = ia[["taxid1"]].copy()
acc2tax.to_parquet(out)  # requires pyarrow or fastparquet
print("saved", out, "rows:", acc2tax.shape[0])


/tmp/ipykernel_144319/1751381301.py:7: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  ia = pd.read_table(


saved tables/accession_taxid.parquet rows: 1962798


In [1]:
import pyarrow.parquet as pq
import pandas as pd
from pathlib import Path

parq = Path("tables/accession_taxid.parquet")
pkl  = Path("tables/accession_taxid.pkl")

# Read parquet using pyarrow (bypasses pandas' extension-type regis


In [2]:
import pyarrow.parquet as pq
import pandas as pd
from pathlib import Path

parq = Path("tables/accession_taxid.parquet")
pkl  = Path("tables/accession_taxid.pkl")

# Read parquet using pyarrow (bypasses pandas' extension-type registration)
table = pq.read_table(parq)
df = table.to_pandas()   # becomes a normal pandas DataFrame

# Make sure accession is the index if it got reset
# (depends on how you wrote it)
if "index" in df.columns and df.index.name is None:
    df = df.set_index("index")

df.to_pickle(pkl)
print("wrote", pkl, "shape:", df.shape)


wrote tables/accession_taxid.pkl shape: (1962798, 1)
